In [10]:
import sys
import os
import pandas as pd

# Add ../code to sys.path (1 level up from demo, into code)
sys.path.append(os.path.abspath("../code"))

from preprocessing_corpus import find_nearest_language_for_softwares,get_authors,get_synonyms_from_file
from fetch_candidates import get_candidate_urls




In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
CZI = pd.read_csv("../corpus/CZI/synonyms_matrix.csv")
candidates_cache_file = "../candidate_urls.json"
synonyms_file = "./synonym_dictionary.json"



In [ ]:
# Add the path to the input file
input_file = "./input.csv"
input_dataframe = pd.read_csv(input_file,delimiter=';')
# Get the synonyms from the file
get_synonyms_from_file(synonyms_file, input_dataframe,CZI_df=CZI)
# Find the nearest language for each software
input_dataframe['language'] = input_dataframe.apply(
    lambda row: find_nearest_language_for_softwares(row['paragraph'], row['name']), axis=1
)
results = input_dataframe['doi'].apply(get_authors)
input_dataframe['authors'] = results.apply(lambda x: ','.join(x.get('authors', [])) if isinstance(x, dict) else '')
# Save the updated DataFrame to a new CSV file
output_file = "./softwares_with_languages.csv"
input_dataframe.to_csv(output_file, index=False)

In [18]:

#Add the candidate URLs to the DataFrame
input_dataframe=pd.read_csv(output_file)
input_dataframe=get_candidate_urls(input_dataframe, candidates_cache_file)
input_dataframe.to_csv(output_file, index=False)

KeyError: 'candidate_urls'